# Import

In [58]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Add configuration file

In [59]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [60]:
from ALL import config 
from util import *

## Set condition

In [61]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [62]:
s3 = S3Manager()

In [63]:
data_type = "AgNewsTitle"

In [64]:
transformer_model = "sentence-transformers/all-mpnet-base-v2"

# Read data

In [65]:
df_path = s3.download(f"Preprocessing/{data_type}/master.csv")

In [66]:
df = pd.read_csv(df_path, index_col=0)

In [67]:
# sampling for test
# df.sample(n=1000, random_state=0)

In [68]:
labels_path = s3.download(f"Preprocessing/{data_type}/class.csv")

In [69]:
with open(labels_path, mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [70]:
max_model_num = config["vectorize"]["sentenceBERT"]["max_model_num"]

# Embedding

In [71]:
def get_sentenceBERT(texts,seed, path):
    model = SentenceTransformer(transformer_model)
    model.save(path)
    vectors = model.encode(texts)
    return vectors

In [ ]:
vectors_path = f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/vector"
models_path = f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/model"

for model_num in tqdm(range(max_model_num)):
    vectors = get_sentenceBERT(
        df.text.tolist(),
        seed=model_num,
        path=make_filepath(f"{models_path}/{model_num}"),
    )

    np.save(
        make_filepath(f"{vectors_path}/raw/{model_num}.npy"),
        np.stack(vectors),
    )

  0%|          | 0/1 [00:00<?, ?it/s]

## upload file

In [ ]:
s3.upload(
    f"../../temporary/Vectorize/{data_type}/sentenceBERT/{transformer_model}/", 
    f"Vectorize/{data_type}/sentenceBERT/{transformer_model}/"
)

In [ ]:
s3.delete_local_all()

In [ ]:
send_line_notify(f"end {data_type} sentenceBERT {transformer_model}")